In [76]:
import pandas as pd
import io
import os

In [77]:
file_path = 'your_file_path_here'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 12 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Name                                                               55 non-null     object 
 1   Project Name                                                       55 non-null     object 
 2   Project Link                                                       55 non-null     object 
 3   Identification of the problem + Understanding gravity of problem   54 non-null     float64
 4   Sizing of the problem                                              55 non-null     int64  
 5   Clarity on product idea/ who they are                              55 non-null     int64  
 6   Understanding of competitors and landscape                         55 non-null     int64  
 7   GTM Strategy (how will they 

In [79]:
print(df.shape)

(55, 12)


In [80]:
# Function to clean and prepare names for matching
def clean_name(name):
    return name.strip().lower()  

In [81]:
!pip install thefuzz python-pptx pymuPDF frontend python-docx

In [82]:
from thefuzz import process
from pptx import Presentation
import fitz #py 3.8 fitz error
from PIL import Image
from docx import Document

In [83]:
# Flatten and clean names from the spreadsheet for matching
spreadsheet_names = [clean_name(name) for entry in df['Name'] for name in entry.split('\n')]

# Fuzzy matching function
def fuzzy_match(file, names):
    highest = process.extractOne(file, names)
    return highest if highest[1] > 80 else None

In [84]:
# Directory where the files are stored
files_dir = '/content/drive/MyDrive/Files'

# Mapping and text extraction process
file_mapping = []
unmatched_files = []
all_files = [file for file in os.listdir(files_dir)]
name_file_mapping = []

for file in all_files:
    # Match files with names from the spreadsheet
    match = fuzzy_match(clean_name(file), spreadsheet_names)
    if match:
        # Search for the full name in the dataframe where match[0] is a substring of the 'Name'
        matched_entry = df[df['Name'].str.contains(match[0], case=False, na=False)]

        # If a matching entry is found in the dataframe
        if not matched_entry.empty:
            for _, row in matched_entry.iterrows():
                name_file_mapping.append({
                    'Name': row['Name'],
                    'file_name': file
                })
        file_mapping.append({'file_name': file, 'matched_name': match[0]})
    else:
        unmatched_files.append(file)



In [87]:
len(file_mapping)

52

In [ ]:
unmatched_files

In [89]:
#Update any missed manual matches
manual_matches = { # unmatched entries }

# Update the file_mapping with manual matches
for file, matched_name in manual_matches.items():
    file_mapping.append({'file_name': file, 'matched_name': clean_name(matched_name)})
    name_file_mapping.append({'Name': matched_name,'file_name': file})

# Now, convert the file mapping list to a DataFrame
file_mapping_df = pd.DataFrame(name_file_mapping)

In [ ]:
file_mapping_df

In [91]:
file_mapping_df=file_mapping_df.drop(37)

In [ ]:
#file_mapping_df=file_mapping_df.drop([30,56,57,58])

file_mapping_df

In [93]:
len(file_mapping_df)

55

In [94]:
# Function to extract text from PPTX files
def extract_text_from_pptx(presentation_path):
    prs = Presentation(presentation_path)
    text = []
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text.append(shape.text.strip())
    return "\n".join(text)


def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)


# Function to extract text from PDF files, including OCR if needed
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page in pdf_document:
        # Extract text
        text += page.get_text()
    pdf_document.close()
    return text

In [95]:
extracted_texts = {}
for mapping in file_mapping:
    file_name = mapping['file_name']
    full_path = os.path.join(files_dir, file_name)
    if full_path.lower().endswith('.pdf'):
        extracted_text = extract_text_from_pdf(full_path)
    elif full_path.lower().endswith('.pptx'):
        extracted_text = extract_text_from_pptx(full_path)
    else:
        continue  # Skip files that are not PDF or PPTX
    extracted_texts[file_name] = extracted_text

In [96]:
len(extracted_texts)

55

In [97]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [98]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]
    # Re-join tokens into a string
    text = ' '.join(tokens)
    return text

# Preprocess all texts and store in a new dictionary
preprocessed_texts = {file_name: preprocess_text(text) for file_name, text in extracted_texts.items()}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [100]:
# Vectorization (example with TF-IDF)
tfidf_vectorizer = TfidfVectorizer(max_features=100) 
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_texts.values())

In [101]:
preprocessed_texts_df = pd.DataFrame.from_dict(extracted_texts, orient='index', columns=['preprocessed_text'])
preprocessed_texts_df['file_name'] = preprocessed_texts_df.index.astype(str)

In [102]:
preprocessed_texts_with_files_df = pd.merge(file_mapping_df, preprocessed_texts_df, on='file_name')

In [ ]:
preprocessed_texts_with_files_df

In [104]:
len(preprocessed_texts_with_files_df)

55

In [105]:
labeled_data_df = pd.read_csv('file_path')
merged_df = pd.merge(labeled_data_df, preprocessed_texts_with_files_df, on='Name')

In [ ]:
merged_df.head(55)

In [107]:
print(merged_df.columns)

Index(['Name', 'Project Name ', 'Project Link',
       'Identification of the problem + Understanding gravity of problem ',
       'Sizing of the problem', 'Clarity on product idea/ who they are ',
       'Understanding of competitors and landscape ',
       'GTM Strategy (how will they reach out to consumers)',
       'Business Model (how will they earn money)', 'Team',
       'Product Differentiator ', 'TOTAL', 'file_name', 'preprocessed_text'],
      dtype='object')


In [ ]:
# Perform an outer join to see which entries are not matching
merged_outer_df = pd.merge(file_mapping_df, labeled_data_df, on='Name', how='outer', indicator=True)

# Check for rows that didn't match in either direction
missing_entries = merged_outer_df[merged_outer_df['_merge'] != 'both']
print(missing_entries)

In [ ]:
# Retrieve the preprocessed text value for the specified name
preprocessed_text_value = preprocessed_texts_with_files_df.loc[preprocessed_texts_with_files_df['Name'].str.contains("Subham Krishna Borah|Aneesh Deshpande|Naela Rafique|Sandeep Sharma", regex=True, na=False), 'preprocessed_text'].values
preprocessed_text_value

In [111]:
merged_df.shape

(55, 14)

In [112]:
num_train_samples = 40
num_test_samples = 15

# Check if we have enough data
if num_train_samples + num_test_samples > len(df):
    raise ValueError("Not enough data to split into training and testing sets as specified.")

# Split the data into training and testing sets
X = merged_df['preprocessed_text']
y = merged_df[['Identification of the problem + Understanding gravity of problem ', 'Sizing of the problem',
               'Clarity on product idea/ who they are ', 'Understanding of competitors and landscape ',
               'GTM Strategy (how will they reach out to consumers)', 'Business Model (how will they earn money)',
               'Team', 'Product Differentiator ']]


In [113]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

In [114]:
# Encode the labels for each score column
encoders = {}
y_encoded = pd.DataFrame()
for col in y.columns:
    le = LabelEncoder()
    y_encoded[col] = le.fit_transform(y[col])
    encoders[col] = le  

In [115]:
y_one_hot = {col: to_categorical(y_encoded[col], num_classes=y_encoded[col].nunique()) for col in y_encoded.columns}

In [116]:
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=num_test_samples, train_size=num_train_samples, random_state=42
)

y_train_one_hot = {col: y_one_hot[col][y_train_encoded.index] for col in y_encoded.columns}
y_test_one_hot = {col: y_one_hot[col][y_test_encoded.index] for col in y_encoded.columns}

In [117]:
tfidf_vectorizer = TfidfVectorizer(max_features=97)  
tfidf_vectorizer.fit(X_train)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [118]:
input_layer = Input(shape=(X_train_tfidf.shape[1],))
dense_layer_1 = Dense(512, activation='relu')(input_layer)
dropout_1 = Dropout(0.5)(dense_layer_1)
dense_layer_2 = Dense(256, activation='relu')(dropout_1)
dropout_2 = Dropout(0.5)(dense_layer_2)


In [119]:
# Output layers: one per score column
output_layers = []
for col in y_train_one_hot:
    num_classes = y_train_one_hot[col].shape[1]
    output_layer = Dense(num_classes, activation='softmax', name=col.replace(' ', '_').replace('+', 'plus').replace('/', '_').replace('(', '').replace(')', ''))(dropout_2)
    output_layers.append(output_layer)

In [120]:

model = Model(inputs=input_layer, outputs=output_layers)

In [121]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], )

In [122]:
model.fit(
    X_train_tfidf.toarray(),
    [y_train_one_hot[key] for key in y_train_one_hot.keys()],  # List of target arrays
    epochs=250,
    batch_size=10
)

Epoch 1/250
4/4 [==============================] - 6s 20ms/step - loss: 8.9486 - Identification_of_the_problem_plus_Understanding_gravity_of_problem__loss: 1.3514 - Sizing_of_the_problem_loss: 1.1218 - Clarity_on_product_idea__who_they_are__loss: 1.0980 - Understanding_of_competitors_and_landscape__loss: 1.0190 - GTM_Strategy_how_will_they_reach_out_to_consumers_loss: 1.1113 - Business_Model_how_will_they_earn_money_loss: 1.0926 - Team_loss: 1.0469 - Product_Differentiator__loss: 1.1076 - Identification_of_the_problem_plus_Understanding_gravity_of_problem__accuracy: 0.4750 - Sizing_of_the_problem_accuracy: 0.2500 - Clarity_on_product_idea__who_they_are__accuracy: 0.3000 - Understanding_of_competitors_and_landscape__accuracy: 0.7000 - GTM_Strategy_how_will_they_reach_out_to_consumers_accuracy: 0.2000 - Business_Model_how_will_they_earn_money_accuracy: 0.3000 - Team_accuracy: 0.5750 - Product_Differentiator__accuracy: 0.3500
Epoch 2/250
4/4 [==============================] - 0s 18ms/step

In [123]:
from sklearn.metrics import accuracy_score

# Predict on the test set
predictions = model.predict(X_test_tfidf.toarray())

# Inverse transform the predictions to get original labels
predicted_labels = {}
for i, key in enumerate(y_train_one_hot.keys()):
    predicted_labels[key] = encoders[key].inverse_transform(np.argmax(predictions[i], axis=1))

# Calculate accuracy for each parameter
accuracies = {}
for key in y_test_encoded:
    true_labels = y_test_encoded[key]
    predicted = predicted_labels[key]
    accuracies[key] = accuracy_score(true_labels, predicted)

# Print accuracies for each parameter
for key, acc in accuracies.items():
    print(f"Accuracy for {key}: {acc * 100:.2f}%")

# Calculate overall accuracy
overall_accuracy = np.mean(list(accuracies.values()))
print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")


1/1 [==============================] - 0s 122ms/step
Accuracy for Identification of the problem + Understanding gravity of problem : 20.00%
Accuracy for Sizing of the problem: 33.33%
Accuracy for Clarity on product idea/ who they are : 20.00%
Accuracy for Understanding of competitors and landscape : 20.00%
Accuracy for GTM Strategy (how will they reach out to consumers): 20.00%
Accuracy for Business Model (how will they earn money): 26.67%
Accuracy for Team: 13.33%
Accuracy for Product Differentiator : 26.67%
Overall Accuracy: 22.50%


In [124]:
# Predict on the test set
predictions = model.predict(X_test_tfidf.toarray())

# Inverse transform the predictions to get original labels
predicted_labels = {key: encoders[key].inverse_transform(np.argmax(predictions[i], axis=1)) for i, key in enumerate(y_train_one_hot.keys())}

# Define the weightages for each parameter
weightages = {
    'Identification of the problem + Understanding gravity of problem ': 0.15,
    'Sizing of the problem': 0.10,
    'Clarity on product idea/ who they are ': 0.10,
    'Understanding of competitors and landscape ': 0.10,
    'GTM Strategy (how will they reach out to consumers)': 0.10,
    'Business Model (how will they earn money)': 0.10,
    'Team': 0.20,
    'Product Differentiator ': 0.15
}

# Function to calculate and print the total weighted score along with individual parameter scores
def calculate_and_print_scores(predicted_labels, weightages):
    for i in range(len(X_test)):
        print(f"Startup {i + 1} Scores:")
        total_score = 0
        for key in predicted_labels:
            # Calculate individual weighted score
            individual_score = predicted_labels[key][i] * weightages[key]
            total_score += individual_score
            print(f"  {key}: {predicted_labels[key][i]} (Weighted Score: {individual_score:.2f})")
        # Print total score
        print(f"  Total Weighted Score: {total_score:.2f}")
        print("")

# Calculate and print the scores for the test set
calculate_and_print_scores(predicted_labels, weightages)


1/1 [==============================] - 0s 23ms/step
Startup 1 Scores:
  Identification of the problem + Understanding gravity of problem : 3.0 (Weighted Score: 0.45)
  Sizing of the problem: 1 (Weighted Score: 0.10)
  Clarity on product idea/ who they are : 3 (Weighted Score: 0.30)
  Understanding of competitors and landscape : 3 (Weighted Score: 0.30)
  GTM Strategy (how will they reach out to consumers): 1 (Weighted Score: 0.10)
  Business Model (how will they earn money): 1 (Weighted Score: 0.10)
  Team: 1 (Weighted Score: 0.20)
  Product Differentiator : 1 (Weighted Score: 0.15)
  Total Weighted Score: 1.70

Startup 2 Scores:
  Identification of the problem + Understanding gravity of problem : 2.0 (Weighted Score: 0.30)
  Sizing of the problem: 1 (Weighted Score: 0.10)
  Clarity on product idea/ who they are : 3 (Weighted Score: 0.30)
  Understanding of competitors and landscape : 1 (Weighted Score: 0.10)
  GTM Strategy (how will they reach out to consumers): 1 (Weighted Score: 0.1

In [125]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_estimators': [10, 50, 100, 200, 500]  
}

# Initialize the model
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Perform Grid Search for each output column separately
best_estimators = {}
for col in y_train_encoded.columns:
    print(f"Tuning hyperparameters for {col}...")
    grid_search.fit(X_train_tfidf, y_train_encoded[col])
    best_estimators[col] = grid_search.best_params_['n_estimators']
    print(f"Best number of estimators for {col}: {best_estimators[col]}")


Tuning hyperparameters for Identification of the problem + Understanding gravity of problem ...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best number of estimators for Identification of the problem + Understanding gravity of problem : 10
Tuning hyperparameters for Sizing of the problem...
Best number of estimators for Sizing of the problem: 200
Tuning hyperparameters for Clarity on product idea/ who they are ...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best number of estimators for Clarity on product idea/ who they are : 50
Tuning hyperparameters for Understanding of competitors and landscape ...
Best number of estimators for Understanding of competitors and landscape : 10
Tuning hyperparameters for GTM Strategy (how will they reach out to consumers)...
Best number of estimators for GTM Strategy (how will they reach out to consumers): 100
Tuning hyperparameters for Business Model (how will they earn money)...
Best number of estimators for Business Model (how will they earn money): 500
Tuning hyperparameters for Team...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best number of estimators for Team: 10
Tuning hyperparameters for Product Differentiator ...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best number of estimators for Product Differentiator : 200


In [126]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train Random Forest models
rf_models = {}
rf_accuracies = {}
for col in y_train_encoded.columns:
    best_n_estimators = best_estimators[col]
    rf = RandomForestClassifier(n_estimators=best_n_estimators, random_state=42)
    rf.fit(X_train_tfidf, y_train_encoded[col])
    rf_models[col] = rf
    y_pred = rf.predict(X_test_tfidf)
    rf_accuracies[col] = accuracy_score(y_test_encoded[col], y_pred)


In [127]:
rf_accuracies

{'Identification of the problem + Understanding gravity of problem ': 0.5333333333333333,
 'Sizing of the problem': 0.5333333333333333,
 'Clarity on product idea/ who they are ': 0.6,
 'Understanding of competitors and landscape ': 0.6,
 'GTM Strategy (how will they reach out to consumers)': 0.4666666666666667,
 'Business Model (how will they earn money)': 0.6666666666666666,
 'Team': 0.8666666666666667,
 'Product Differentiator ': 0.4}

In [128]:
from sklearn.ensemble import GradientBoostingClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100, 200, 500]  # You can choose a different range of values
}

gb = GradientBoostingClassifier(random_state=42)

grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

best_estimators = {}
for col in y_train_encoded.columns:
    print(f"Tuning hyperparameters for {col}...")
    grid_search.fit(X_train_tfidf, y_train_encoded[col])
    best_estimators[col] = grid_search.best_params_['n_estimators']
    print(f"Best number of estimators for {col}: {best_estimators[col]}")


Tuning hyperparameters for Identification of the problem + Understanding gravity of problem ...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best number of estimators for Identification of the problem + Understanding gravity of problem : 10
Tuning hyperparameters for Sizing of the problem...
Best number of estimators for Sizing of the problem: 10
Tuning hyperparameters for Clarity on product idea/ who they are ...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best number of estimators for Clarity on product idea/ who they are : 200
Tuning hyperparameters for Understanding of competitors and landscape ...
Best number of estimators for Understanding of competitors and landscape : 10
Tuning hyperparameters for GTM Strategy (how will they reach out to consumers)...
Best number of estimators for GTM Strategy (how will they reach out to consumers): 10
Tuning hyperparameters for Business Model (how will they earn money)...
Best number of estimators for Business Model (how will they earn money): 50
Tuning hyperparameters for Team...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best number of estimators for Team: 10
Tuning hyperparameters for Product Differentiator ...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best number of estimators for Product Differentiator : 10


In [129]:
from sklearn.ensemble import GradientBoostingClassifier

# Train Gradient Boosting models
gb_models = {}
gb_accuracies = {}
for col in y_train_encoded.columns:
    best_n_estimators = best_estimators[col]
    gb = GradientBoostingClassifier(n_estimators=best_n_estimators, random_state=42)
    gb.fit(X_train_tfidf, y_train_encoded[col])
    gb_models[col] = gb
    y_pred = gb.predict(X_test_tfidf)
    gb_accuracies[col] = accuracy_score(y_test_encoded[col], y_pred)


In [130]:
gb_accuracies

{'Identification of the problem + Understanding gravity of problem ': 0.6,
 'Sizing of the problem': 0.5333333333333333,
 'Clarity on product idea/ who they are ': 0.6666666666666666,
 'Understanding of competitors and landscape ': 0.6,
 'GTM Strategy (how will they reach out to consumers)': 0.4666666666666667,
 'Business Model (how will they earn money)': 0.6,
 'Team': 0.8666666666666667,
 'Product Differentiator ': 0.6}

In [131]:
#ensemble: averaging the predictions
ensemble_predictions = {}
ensemble_accuracies = {}
for col in y_train_encoded.columns:
    rf_pred = rf_models[col].predict_proba(X_test_tfidf)
    gb_pred = gb_models[col].predict_proba(X_test_tfidf)
    avg_pred = (rf_pred + gb_pred) / 2
    ensemble_predictions[col] = np.argmax(avg_pred, axis=1)
    ensemble_accuracies[col] = accuracy_score(y_test_encoded[col], ensemble_predictions[col])


In [132]:
ensemble_accuracies

{'Identification of the problem + Understanding gravity of problem ': 0.6666666666666666,
 'Sizing of the problem': 0.4666666666666667,
 'Clarity on product idea/ who they are ': 0.6666666666666666,
 'Understanding of competitors and landscape ': 0.6,
 'GTM Strategy (how will they reach out to consumers)': 0.6,
 'Business Model (how will they earn money)': 0.6666666666666666,
 'Team': 0.8666666666666667,
 'Product Differentiator ': 0.6}

In [133]:
# accuracy for Random Forest
overall_rf_accuracy = np.mean(list(rf_accuracies.values()))
print(f"Overall Random Forest Accuracy: {overall_rf_accuracy:.2%}")

# accuracy for Gradient Boosting
overall_gb_accuracy = np.mean(list(gb_accuracies.values()))
print(f"Overall Gradient Boosting Accuracy: {overall_gb_accuracy:.2%}")

#accuracy for the ensemble method
ensemble_correct_predictions = np.sum([y_test_encoded[col] == ensemble_predictions[col] for col in y_test_encoded.columns], axis=0)
ensemble_overall_accuracy = np.mean(ensemble_correct_predictions / len(y_test_encoded))
print(f"Overall Ensemble Accuracy: {ensemble_overall_accuracy:.2%}")

# neural net accuracy
overall_accuracy = np.mean(list(accuracies.values()))
print(f"Overall Neural Network Accuracy: {overall_accuracy * 100:.2f}%")


Overall Random Forest Accuracy: 58.33%
Overall Gradient Boosting Accuracy: 61.67%
Overall Ensemble Accuracy: 34.22%
Overall Neural Network Accuracy: 22.50%


In [134]:
def predict_and_calculate_total(file_path, tfidf_vectorizer, model, encoders, weightages):
    # Check file extension and extract text
    if file_path.lower().endswith('.pdf'):
        extracted_text = extract_text_from_pdf(file_path)
    elif file_path.lower().endswith('.pptx'):
        extracted_text = extract_text_from_pptx(file_path)
    elif file_path.lower().endswith('.docx'):
        extracted_text = extract_text_from_docx(file_path)
    else:
        print("Unsupported file format. Please provide a PDF or PPTX file.")
        return

    # Preprocess the text
    preprocessed_text = preprocess_text(extracted_text)

    # Transform the text to TF-IDF features
    tfidf_features = tfidf_vectorizer.transform([preprocessed_text]).toarray()

    # Predict the scores
    predictions = model.predict(tfidf_features)

    # Decode predictions if necessary
    predicted_labels = {}
    for i, col in enumerate(encoders.keys()):
        label_class = encoders[col].inverse_transform([np.argmax(predictions[i], axis=-1)])[0]
        predicted_labels[col] = label_class

    # Calculate the total score
    total_score = 0
    print("Predicted Parameter Scores:")
    for key, score in predicted_labels.items():
        # If the scores are not already in a scaled format, you may need to scale them accordingly
        individual_weighted_score = score * weightages[key]
        total_score += individual_weighted_score
        print(f"  {key}: {score} (Weighted Score: {individual_weighted_score})")
    total_score_rounded = round(total_score, 2)
    print(f"\nTotal Weighted Score: {total_score_rounded}")
    return predicted_labels, total_score_rounded

Randon forest model: rf
Gradient Boosting: gb
Neural Network: model

In [135]:
file_to_predict = 'file_path'  # Change this to the path of the file you want to predict
predict_and_calculate_total(file_to_predict, tfidf_vectorizer, model, encoders, weightages)

1/1 [==============================] - 0s 21ms/step
Predicted Parameter Scores:
  Identification of the problem + Understanding gravity of problem : 3.0 (Weighted Score: 0.44999999999999996)
  Sizing of the problem: 2 (Weighted Score: 0.2)
  Clarity on product idea/ who they are : 2 (Weighted Score: 0.2)
  Understanding of competitors and landscape : 1 (Weighted Score: 0.1)
  GTM Strategy (how will they reach out to consumers): 2 (Weighted Score: 0.2)
  Business Model (how will they earn money): 2 (Weighted Score: 0.2)
  Team: 1 (Weighted Score: 0.2)
  Product Differentiator : 2 (Weighted Score: 0.3)

Total Weighted Score: 1.85


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

({'Identification of the problem + Understanding gravity of problem ': 3.0,
  'Sizing of the problem': 2,
  'Clarity on product idea/ who they are ': 2,
  'Understanding of competitors and landscape ': 1,
  'GTM Strategy (how will they reach out to consumers)': 2,
  'Business Model (how will they earn money)': 2,
  'Team': 1,
  'Product Differentiator ': 2},
 1.85)

In [136]:
model.save('my_keras_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [137]:
import joblib

# Save the model
joblib.dump(gb, 'gradient_boosting_model.joblib')


['gradient_boosting_model.joblib']